# MODEL EVALUATION

In [1]:
import os 
%pwd

"c:\\Users\\soulo\\MACHINE_LEARNING_UNIVERSE\\PROJECT'S\\End-to-End-Machine-Leaning-Project-With-Mlow\\research"

In [2]:
os.chdir('../')
%pwd

"c:\\Users\\soulo\\MACHINE_LEARNING_UNIVERSE\\PROJECT'S\\End-to-End-Machine-Leaning-Project-With-Mlow"